In [0]:
CREATE OR REFRESH LIVE TABLE dim_date_da AS
SELECT * FROM LIVE.silver_date;

In [0]:
CREATE OR REFRESH LIVE TABLE dim_geo_da AS
WITH src AS (
  SELECT DISTINCT
    zip_code,
    city,
    state
  FROM LIVE.silver_geo
)
SELECT
  DENSE_RANK() OVER (ORDER BY zip_code, city, state) AS location_key,
  zip_code,
  city,
  state
FROM src;

In [0]:
CREATE OR REFRESH LIVE TABLE dim_inspection_type_da AS
SELECT * FROM LIVE.silver_inspection_type

In [0]:
CREATE OR REFRESH LIVE TABLE dim_violation_da AS
SELECT * FROM LIVE.silver_violation

In [0]:
CREATE OR REFRESH LIVE TABLE fact_inspection_da AS
WITH src AS (
  SELECT
    restaurant_name,
    inspection_type,
    CAST(inspection_date AS DATE) AS inspection_dt,
    inspection_score,
    total_violation_score,
    street_address,
    zip_code,
    latitude,
    longitude,
    load_dt,
    source_file_path,
    source_file_name
  FROM LIVE.silver_food_inspection_dallas
),

biz AS (
  SELECT
    business_key,          
    restaurant_name,
    effective_start_dt,
    effective_end_dt
  FROM LIVE.dim_business_da_scd2_v2
),

dt AS (
  SELECT
    date_key,
    full_date
  FROM LIVE.dim_date_da            
),

insp_type AS (
  SELECT
    inspection_type_key,
    inspection_type
  FROM LIVE.dim_inspection_type_da 
),

loc AS (
  SELECT
    location_key,
    zip_code,
    city,
    state
  FROM LIVE.dim_geo_da       
)

SELECT
  ROW_NUMBER() OVER (
    ORDER BY s.restaurant_name, s.inspection_dt, s.inspection_type
  )                       AS inspection_da_key,

  d.date_key              AS date_key,
  b.business_key          AS business_da_key,     
  l.location_key          AS location_key,
  it.inspection_type_key  AS inspection_type_key,

  s.inspection_score,
  s.total_violation_score,
  s.load_dt,
  s.source_file_path,
  s.source_file_name,

  -- keep raw fields for traceability / bridge join
  s.restaurant_name,
  s.inspection_dt    AS inspection_date,
  s.inspection_type,
  s.zip_code
FROM src s

LEFT JOIN dt d
  ON d.full_date = s.inspection_dt

LEFT JOIN biz b
  ON trim(s.restaurant_name) = trim(b.restaurant_name)

LEFT JOIN loc l
  ON s.zip_code = l.zip_code

LEFT JOIN insp_type it
  ON s.inspection_type = it.inspection_type;

In [0]:
CREATE OR REFRESH LIVE TABLE bridge_inspection_violation_da AS
WITH src AS (
  SELECT
    f.inspection_da_key,
    s.violation_description_1,
    s.violation_description_2,
    s.violation_description_3,
    s.violation_description_4,
    s.violation_description_5,
    s.violation_description_6,
    s.violation_description_7,
    s.violation_description_8,
    s.violation_description_9,
    s.violation_description_10,
    s.violation_description_11,
    s.violation_description_12,
    s.violation_description_13,
    s.violation_description_14,
    s.violation_description_15,
    s.violation_description_16,
    s.violation_description_17,
    s.violation_description_18,
    s.violation_description_19,
    s.violation_description_20,
    s.violation_description_21,
    s.violation_description_22,
    s.violation_description_23,
    s.violation_description_24,
    s.violation_description_25
  FROM LIVE.silver_food_inspection_dallas s
  JOIN LIVE.fact_inspection_da f
    ON  s.load_dt          = f.load_dt
    AND s.source_file_path = f.source_file_path
    AND s.source_file_name = f.source_file_name
),

exploded AS (
  SELECT
    inspection_da_key,
    posexplode(
      array(
        violation_description_1,
        violation_description_2,
        violation_description_3,
        violation_description_4,
        violation_description_5,
        violation_description_6,
        violation_description_7,
        violation_description_8,
        violation_description_9,
        violation_description_10,
        violation_description_11,
        violation_description_12,
        violation_description_13,
        violation_description_14,
        violation_description_15,
        violation_description_16,
        violation_description_17,
        violation_description_18,
        violation_description_19,
        violation_description_20,
        violation_description_21,
        violation_description_22,
        violation_description_23,
        violation_description_24,
        violation_description_25
      )
    ) AS (pos, violation_description)
  FROM src
),

flat AS (
  SELECT
    inspection_da_key,
    pos + 1 AS sequence_n,     -- 1..25
    violation_description,
    -- extract numeric code from descriptions like "*24 Food Labeling-..."
    regexp_extract(violation_description, '(\\d+)', 1) AS violation_code
  FROM exploded
  WHERE violation_description IS NOT NULL
),

joined AS (
  SELECT
    f.inspection_da_key,
    v.violation_code,
    f.sequence_n
  FROM flat f
  JOIN LIVE.dim_violation_da v
    ON v.violation_code = f.violation_code
)

SELECT DISTINCT
  inspection_da_key,
  violation_code,
  sequence_n
FROM joined;